In [6]:
import os
import numpy as np
import pandas as pd

#Config
root = r"C:\Users\Glen\Documents\School\Scratch\Semetser_Project" #replace this with the folder you've stored the files in

FYE_file = "UNO_Student_Experience_Survey_1"
NFYE_file = "UNO_Student_Experience_Survey_2"

FYE_path = os.path.join(root, FYE_file)
NFYE_path = os.path.join(root, NFYE_file)

#load and clean
def load_csv(path, group_label, min_progress=0):
    df = pd.read_csv(path)

    # Drop the first two rows:
    #   row 0 = question text
    #   row 1 = ImportId metadata
    df = df.iloc[2:, :].reset_index(drop=True)

    # Basic type cleaning
    if "Progress" in df.columns:
        df["Progress"] = pd.to_numeric(df["Progress"], errors="coerce")
    if "Duration (in seconds)" in df.columns:
        df["Duration (in seconds)"] = pd.to_numeric(
            df["Duration (in seconds)"], errors="coerce")

    # Filter to finished responses only
    if "Finished" in df.columns:
        df = df[df["Finished"] == "True"]

    # Min progress threshold
    # min_progress=0 to ignore
    if "Progress" in df.columns and min_progress is not None:
        df = df[df["Progress"] >= min_progress]

    df["group"] = group_label

    return df

#Recode ordinal and binary scales to numeric
# add map for demographics
def recode_scales(df):
    agree_map = {
        "Strongly disagree.": 1,
        "Disagree.": 2,
        "Neutral.": 3,
        "Agree.": 4,
        "Strongly agree.": 5}
    agree_set = set(agree_map.keys())

    connected_map = {
        "Not connected at all.": 1,
        "Neutral.": 2,
        "Somewhat connected.": 3,
        "Connected.": 4}
    connected_set = set(connected_map.keys())
        
    freq_map = {
        "0": 0,
        "1": 1,
        "2": 2,
        "3": 3,
        "4+": 4}
    freq_set = set(freq_map.keys())

    yesno_map = {
        "No.": 0,
        "Yes.": 1,
        "No": 0,
        "Yes": 1,
        "Prefer not to say": np.nan,
        "Prefer not to say.": np.nan}
    yesno_set = set(k for k in yesno_map.keys() if yesno_map[k] in [0,1])

    gpa_map = {
        "2.5 or lower": 1,
        "2.5-3.0": 2,
        "3.0-3.25": 3,
        "3.25-3.5": 4,
        "3.5-3.75": 5,
        "3.75+": 6}
    gpa_set = set(gpa_map.keys())

    hours_map = {
        "5 or fewer hours.": 1,
        "6 - 10 hours.": 2,
        "11 - 15 hours.": 3,
        "16 - 20 hours.": 4,
        "21 - 25 hours.": 5,
        "Greater than 25 hours.": 6}
    hours_set = set(hours_map.keys())

    change_map = {
        # Academic performance
        "My grades have improved on average.": 3,
        "My grades have stayed the same on average.": 2,
        "My grades have worsened on average.": 1,

        # Time management skills
        "My time management skills have improved.": 3,
        "My time management skills have stayed the same.": 2,
        "My time management skills have worsened.": 1,

        # Study habits
        "I study more than before.": 3,
        "My study habits have not changed.": 2,
        "I study less than before.": 1}
    change_set = set(change_map.keys())



    for col in df.columns:
        # Only bother recoding question columns
        if not col.startswith("Q"):
            continue

        vals = set(df[col].dropna().unique())
        if not vals:
            continue

        if vals.issubset(agree_set):
            new_col = col + "_num"
            df[new_col] = df[col].map(agree_map).astype("float")

        elif vals.issubset(connected_set):
            new_col = col + "_num"
            df[new_col] = df[col].map(connected_map).astype("float")

        elif vals.issubset(freq_set):
            new_col = col + "_num"
            df[new_col] = df[col].map(freq_map).astype("float")

        elif vals.issubset(yesno_set):
            new_col = col + "_num"
            df[new_col] = df[col].map(yesno_map).astype("float")

        elif vals.issubset(gpa_set):
            # "What is your current GPA?"
            new_col = col + "_num"
            df[new_col] = df[col].map(gpa_map).astype("float")

        elif vals.issubset(hours_set):
            # "About how many hours per week do you spend on academic work outside of class?"
            new_col = col + "_num"
            df[new_col] = df[col].map(hours_map).astype("float")

        elif vals.issubset(change_set):
            # Change questions: grades, time management, study habits
            new_col = col + "_num"
            df[new_col] = df[col].map(change_map).astype("float")

    return df



#Score financial literacy items Q37–Q41
def score_financial_literacy(df):
    def binary_score(series, correct_answer):
        return np.where(
            series.isna(), np.nan,
            np.where(series == correct_answer, 1, 0)
        )

    if "Q37" in df.columns:
        df["Q37_correct"] = binary_score(df["Q37"], "More than $102.")
    if "Q38" in df.columns:
        df["Q38_correct"] = binary_score(df["Q38"], "Less than today.")
    if "Q39" in df.columns:
        df["Q39_correct"] = binary_score(df["Q39"], "They will fall.")
    if "Q40" in df.columns:
        df["Q40_correct"] = binary_score(df["Q40"], "True.")
    if "Q41" in df.columns:
        df["Q41_correct"] = binary_score(df["Q41"], "False.")

    fin_cols = [c for c in df.columns if c.endswith("_correct")]
    if fin_cols:
        df["FinLit_Total"] = df[fin_cols].sum(axis=1)

    return df

#Master function - prepare combined dataset for analysis
def prepare_analysis_dataset(FYE_path, NFYE_path, min_progress=0):
    fye = load_qualtrics_csv(FYE_path, group_label="FYE", min_progress=min_progress)
    nfye = load_qualtrics_csv(NFYE_path, group_label="NonFYE", min_progress=min_progress)

    # Make sure both have same columns (they should, but just in case)
    common_cols = fye.columns.intersection(nfye.columns)
    fye = fye[common_cols].copy()
    nfye = nfye[common_cols].copy()

    # Recode scales and score financial literacy, separately
    fye = recode_scales(fye)
    nfye = recode_scales(nfye)

    fye = score_financial_literacy(fye)
    nfye = score_financial_literacy(nfye)

    combined = pd.concat([fye, nfye], ignore_index=True)

    return combined


In [ ]:
#run Mann Whitney for questions on connectedness, engaement, time management, etc
if __name__ == "__main__":
    combined = prepare_analysis_dataset(FYE_path, NFYE_path, min_progress=0)